In [1]:
from _results_helper import *
import warnings
warnings.filterwarnings("ignore")

choices = [
    # "sample",
    "best",
]
layers = [
    # 1,
    2,
    3,
    4,
    # 5,
]
dimensions = [
    # 1,
    # 2,
    # 4,
    8,
    16,
    32,
    # 64,
    # 128,
]

this_file_dir=PosixPath('/home/dzc/code/cvut-colab/results')
Raw logs found. Rewriting data to csv files...


/home/dzc/code/cvut-colab/results/baseline_logs/satellite_1_30_r2.log: 100%|██████████| 923/923 [00:00<00:00, 1646.61it/s]  


skipping hard problems because they take too long
Training failure in /home/dzc/code/cvut-colab/results/train_logs/satellite_3_8_0.log
Training failure in /home/dzc/code/cvut-colab/results/train_logs/satellite_3_8_2.log
Training failure in /home/dzc/code/cvut-colab/results/train_logs/satellite_3_16_0.log
Training failure in /home/dzc/code/cvut-colab/results/train_logs/satellite_3_16_1.log


# Train Stats

In [2]:
visualise_train()

blocksworld


ferry


satellite


rover


# Plan length plots

In [3]:
plot_domains("plan_length", log_y=0, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover


# Cycle plots

In [4]:
plot_domains("cycles", log_y=0, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover


# Quantified improvements

In [5]:
quantify_performance(choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover


IndexError: index 0 is out of bounds for axis 0 with size 0

# Relative difference plots

In [ ]:
plot_difference(absolute=False, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover


IndexError: index 0 is out of bounds for axis 0 with size 0